In [1]:
# Import the necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import requests

In [2]:
# Make a GET request to the specified URL
url = "https://api.beta.ons.gov.uk/v1/datasets"

response = requests.get(url)

# Print the response as JSON
response.json()

{'items': [{'contacts': [{'email': 'qualityoflife@ons.gov.uk',
     'name': 'Will Shufflebottom, Owain Birrell, Pavan Bains and Geeta Kerai',
     'telephone': '+44 300 0671543'}],
   'description': 'Seasonally and non seasonally-adjusted quarterly estimates of life satisfaction, feeling that the things done in life are worthwhile, happiness and anxiety in the UK.',
   'keywords': ['well-being'],
   'id': 'wellbeing-quarterly',
   'last_updated': '2023-12-13T09:40:24.204Z',
   'links': {'editions': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/wellbeing-quarterly/editions'},
    'latest_version': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/wellbeing-quarterly/editions/time-series/versions/9',
     'id': '9'},
    'self': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/wellbeing-quarterly'},
    'taxonomy': {'href': 'https://api.beta.ons.gov.uk/v1/peoplepopulationandcommunity/wellbeing'}},
   'methodologies': [{'href': 'https://www.ons.gov.uk/peoplepopulationandcommunity/wel

In [3]:
ons_url = "https://api.beta.ons.gov.uk/v1/datasets"
data_set = "uk-spending-on-cards"

In [6]:
def get_latest_version(data_set, root_url = "https://api.beta.ons.gov.uk/v1/datasets"):
    """Fetch the latest version information for a given dataset from the ONS API."""
    url = f"{root_url}/{data_set}"
    response = requests.get(url)
    latest_url = response.json()["links"]["latest_version"]["href"]
    print(latest_url)
    response = requests.get(latest_url)
    return response.json(), latest_url

In [9]:
get_latest_version(data_set)[0]
url_to_use = get_latest_version(data_set)[1]

https://api.beta.ons.gov.uk/v1/datasets/uk-spending-on-cards/editions/time-series/versions/130
https://api.beta.ons.gov.uk/v1/datasets/uk-spending-on-cards/editions/time-series/versions/130


In [19]:
def find_dimensions(url):
    response = requests.get(url + "/dimensions")

    items = response.json().get("items")

    dims = {}

    for item in items:
        id = item['links']['options']['id']
        options_url = f"{url}/dimensions/{id}/options"
        option_response = requests.get(options_url)
        option = option_response.json()
        option_descriptions = {
            item["option"]: item['label'] for item in option['items']
        }
        dims[id] = option_descriptions

    return(dims)

In [23]:
valid_dims = find_dimensions(url_to_use)

valid_dims

{'category': {'aggregate': 'Aggregate',
  'delayable': 'Delayable',
  'social': 'Social',
  'staple': 'Staple',
  'work-related': 'Work Related'},
 'daymonth': {'31-12': '31-12',
  '30-12': '30-12',
  '29-12': '29-12',
  '28-12': '28-12',
  '27-12': '27-12',
  '24-12': '24-12',
  '23-12': '23-12',
  '22-12': '22-12',
  '21-12': '21-12',
  '20-12': '20-12',
  '19-12': '19-12',
  '18-12': '18-12',
  '17-12': '17-12',
  '16-12': '16-12',
  '15-12': '15-12',
  '14-12': '14-12',
  '13-12': '13-12',
  '12-12': '12-12',
  '11-12': '11-12',
  '10-12': '10-12'},
 'geography': {'K02000001': 'United Kingdom'},
 'time': {'2024': '2024',
  '2023': '2023',
  '2022': '2022',
  '2021': '2021',
  '2020': '2020'}}

In [114]:
selected_request = {'category' : 'aggregate', 'daymonth' : '31-01', 'geography': 'K02000001', 'time': '*'}

response = requests.get(url_to_use + "/observations", params = selected_request)

observations = response.json()['observations']

variable = [key for key, value in selected_request.items() if value == '*'][0]

summary = []
for observation in observations:
    id = next(iter(observation['dimensions'].values()))['label']
    summary.append({variable: id, "observation": observation["observation"]})
df = pd.DataFrame(summary)

for name in selected_request.keys():
    if (name in df.columns):
        next
    else:
        df[name] = selected_request[name]

df

,time,observation,category,daymonth,geography
0,2023,98.5101710694974,aggregate,31-01,K02000001
1,2020,96.80340921324336,aggregate,31-01,K02000001
2,2021,,aggregate,31-01,K02000001
3,2024,93.19806454014902,aggregate,31-01,K02000001
4,2022,90.4738864099284,aggregate,31-01,K02000001


In [112]:
#variable = [key for key, value in selected_request.items() if value == '*']

selected_request['category']

'aggregate'

In [ ]:
for observation in observations:
    print(observation['dimensions'])

{'dimensions': {'DayMonth': {'href': 'http://10.30.156.182:10500/code-lists/dd-mm/codes/14-01', 'id': '14-01', 'label': '14-01'}}, 'metadata': {'Data Marking': '.'}, 'observation': ''}
{'dimensions': {'DayMonth': {'href': 'http://10.30.156.182:10500/code-lists/dd-mm/codes/31-01', 'id': '31-01', 'label': '31-01'}}, 'metadata': {'Data Marking': ''}, 'observation': '98.5101710694974'}
{'dimensions': {'DayMonth': {'href': 'http://10.30.156.182:10500/code-lists/dd-mm/codes/03-08', 'id': '03-08', 'label': '03-08'}}, 'metadata': {'Data Marking': ''}, 'observation': '101.23761263079355'}
{'dimensions': {'DayMonth': {'href': 'http://10.30.156.182:10500/code-lists/dd-mm/codes/19-01', 'id': '19-01', 'label': '19-01'}}, 'metadata': {'Data Marking': ''}, 'observation': '91.75308935777136'}
{'dimensions': {'DayMonth': {'href': 'http://10.30.156.182:10500/code-lists/dd-mm/codes/15-02', 'id': '15-02', 'label': '15-02'}}, 'metadata': {'Data Marking': ''}, 'observation': '96.23550461745039'}
{'dimension